# Streaming and collecting real-time candles

{
  "e": "kline",     // Event type
  "E": 123456789,   // Event time
  "s": "BNBBTC",    // Symbol
  "k": {
    "t": 123400000, // Kline start time
    "T": 123460000, // Kline close time
    "s": "BNBBTC",  // Symbol
    "i": "1m",      // Interval
    "f": 100,       // First trade ID
    "L": 200,       // Last trade ID
    "o": "0.0010",  // Open price
    "c": "0.0020",  // Close price
    "h": "0.0025",  // High price
    "l": "0.0015",  // Low price
    "v": "1000",    // Base asset volume
    "n": 100,       // Number of trades
    "x": false,     // Is this kline closed?
    "q": "1.0000",  // Quote asset volume
    "V": "500",     // Taker buy base asset volume
    "Q": "0.500",   // Taker buy quote asset volume
    "B": "123456"   // Ignore
  }
}

In [1]:
api_key = "jiaHs9nkNEzJMjNC6XtICaQMx1reQqRQapYn3LsoJeRkWpZAFwuIrSoZuDmH9O9g"
secret_key = "w7z2c13PpmK4e7Y7eiwg81dCB83BHuO2bZEQNq65EQB5ByHFPABEPSamwtdxYNj2"

In [2]:
import pandas as pd #import pandas
from binance import ThreadedWebsocketManager
from binance.client import Client

In [3]:
client = Client(api_key = api_key, api_secret = secret_key, tld = "com")

In [4]:
twm = ThreadedWebsocketManager()
twm.start()

In [5]:
df = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Complete"])
df

,Open,High,Low,Close,Volume,Complete


In [6]:
def stream_candles(msg):
    #extract the required message from msg
    event_time = pd.to_datetime(msg["E"], unit = "ms")
    start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
    first    = float(msg["k"]["o"])
    high     = float(msg["k"]["h"])
    low      = float(msg["k"]["l"])
    close    = float(msg["k"]["c"])
    volume   = float(msg["k"]["v"])
    complete =       msg["k"]["x"]
    
    print("Time: {} | Price: {}".format(event_time, close))
          
    #feed df (add new bar or update latest bar to dataframe above)
    df.loc[start_time] = [first, high, low, close, volume, complete]

In [7]:
twm.start_kline_socket(callback = stream_candles, symbol = "BTCUSDT", interval = "1m")

'btcusdt@kline_1m'

Time: 2022-02-02 09:11:40.928000 | Price: 38407.85
Time: 2022-02-02 09:11:42.944000 | Price: 38410.88
Time: 2022-02-02 09:11:44.975000 | Price: 38416.65
Time: 2022-02-02 09:11:47.359000 | Price: 38421.82
Time: 2022-02-02 09:11:49.956000 | Price: 38421.81
Time: 2022-02-02 09:11:52.126000 | Price: 38421.77
Time: 2022-02-02 09:11:54.974000 | Price: 38410.29
Time: 2022-02-02 09:11:57.258000 | Price: 38401.08
Time: 2022-02-02 09:11:59.734000 | Price: 38404.46
Time: 2022-02-02 09:12:00.002000 | Price: 38404.47
Time: 2022-02-02 09:12:02.296000 | Price: 38402.32
Time: 2022-02-02 09:12:04.907000 | Price: 38394.38
Time: 2022-02-02 09:12:07.001000 | Price: 38402.33
Time: 2022-02-02 09:12:09.126000 | Price: 38402.33
Time: 2022-02-02 09:12:11.148000 | Price: 38393.3
Time: 2022-02-02 09:12:14.031000 | Price: 38393.29
Time: 2022-02-02 09:12:16.066000 | Price: 38393.3
Time: 2022-02-02 09:12:18.125000 | Price: 38392.93
Time: 2022-02-02 09:12:21.068000 | Price: 38388.0
Time: 2022-02-02 09:12:23.809000 |

In [9]:
twm.stop()

Time: 2022-02-02 09:13:13.703000 | Price: 38431.42


In [10]:
df

,Open,High,Low,Close,Volume,Complete
2022-02-02 09:11:00,38439.21,38439.21,38401.08,38404.47,12.00705,True
2022-02-02 09:12:00,38404.47,38427.09,38388.00,38427.08,7.70179,True
2022-02-02 09:13:00,38427.08,38431.42,38414.43,38431.42,3.56682,False


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3 entries, 2022-02-02 09:11:00 to 2022-02-02 09:13:00
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Open      3 non-null      float64
 1   High      3 non-null      float64
 2   Low       3 non-null      float64
 3   Close     3 non-null      float64
 4   Volume    3 non-null      float64
 5   Complete  3 non-null      object 
dtypes: float64(5), object(1)
memory usage: 276.0+ bytes
